142行目「os.system(cmd)」をコメントアウト. 意味のない繰り返しだったため.

anacondaからpythonを入手した場合以下が必要（KernelからRの設定方法）：  
conda install -c r r-modeltools  
conda install -c r r-mvtnorm  
conda install -c r r-coin 

11行目「python = '~'」をコメントアウト.

29行目 ref = reflist[0]　よりreflist選択可能に

In [1]:
## micca is required. pip install micca ##

import os, glob, itertools
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [162]:
#### directories ####
proj_name = 'PRJEB9524'
input_dir = '/Volumes/BUFFALO_4T/tanamai/16S/PRJEB9524/'
output_dir = input_dir
#python = '/System/Library/Frameworks/Python.framework/Versions/2.7/Resources/Python.app/Contents/MacOS/Python'
keggc = '/Users/petadimensionlab/ws/apps/16S/HUMAnN2/HUMAnN1-data/keggc'
LEfSe = '/Users/petadimensionlab/ws/apps/16S/LEfSe/'
graphlan = '/Users/petadimensionlab/ws/apps/16S/graphlan/'

# GreenGenes #
gg97fasta = '/Users/petadimensionlab/ws/ref/gg_13_5_otus/rep_set/97_otus.fasta'
gg97tax = '/Users/petadimensionlab/ws/ref/gg_13_5_otus/taxonomy/97_otu_taxonomy.txt'
# Silva #
silva_fasta = '/Users/petadimensionlab/ws/ref/silva_128/rep_set/97_otus_16S.fasta'
silva_tax = '/Users/petadimensionlab/ws/ref/silva_128/taxonomy/consensus_taxonomy_all_levels.txt'
# RDP #
rdp_fasta = '/Users/petadimensionlab/ws/ref/trainset16_022016.rdp/trainset16_022016.rdp.fasta'
rdp_tax = '/Users/petadimensionlab/ws/ref/trainset16_022016.rdp/trainset16_022016.rdp.tax'
class_def = os.path.join(input_dir,'class_def.txt')

fastalist = { 'gg':gg97fasta,'silva':silva_fasta,'rdp':rdp_fasta }
taxlist = { 'gg':gg97tax,'silva':silva_tax,'rdp':rdp_tax }
header = { 'gg':'gg','silva':'silva','rdp':'rdp' }

reflist = ['gg','silva','rdp']
ref = reflist[0]

In [163]:
is_eps = 'yes'
is_convert_biom = 'no'
is_micca2picrust = 'yes'
is_exec_picrust = 'no'
is_exec_HUMAnN2 = 'no'
is_exec_LEfSe = 'no'
is_exec_graphlan = 'no'

In [164]:
tmp_dir = os.path.join(input_dir,'tmp')
if not os.path.exists(tmp_dir):
    os.makedirs(tmp_dir)
tmp2_dir = os.path.join(tmp_dir,'tmp2')
if not os.path.exists(tmp2_dir):
    os.makedirs(tmp2_dir)
output_dir = os.path.join(input_dir,'output')
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

In [165]:
def readtable(input_fn):
    """Read OTU/Taxa tables."""
    table = pd.read_csv(input_fn, sep='\t', index_col=0)
    # cast index into string
    table.index = [str(elem) for elem in table.index]
    return table

In [166]:
## modified to python 3, see for the error and a way to repair ##
## https://stackoverflow.com/questions/5237611/itertools-cycle-next ##

def plotbars(input_fn, output_fn, raw=False, topn=12, xticklabelsize=8, fmt="png"):
    """plt.gcf().canvas.get_supported_filetypes() """
    table = readtable(input_fn)
    # if raw is False, plot relative abundances
    if not raw:
        table = table / table.sum(axis=0)

    if topn > table.shape[0]:
        topn = table.shape[0]
    # sort by taxa abundances in decreasing order
    taxsum = table.sum(axis=1)
    taxsum.sort_values(ascending=False, inplace=True)
    table = table.reindex(taxsum.index)
    # sort by sample names
    table.sort_index(axis=1, inplace=True)
    # colors, hatches and edgecolors iterators
    ncolors = 12
    colors = [plt.cm.Paired(i) for i in np.linspace(0, 1, ncolors)]
    edgecolors = ['k', 'w']
    hatches = [None]*ncolors + ['////']*ncolors + ['xxxx']*ncolors
    itercolors = itertools.cycle(colors)
    iterhatches = itertools.cycle(hatches)
    iteredgecolors = itertools.cycle(edgecolors)
    # custom rc. "svg.fonttype: none" corrects the conversion of text in PDF
    # and SVG files
    rc = {"svg.fonttype": "none"}
    with plt.rc_context(rc=rc):
        fig = plt.figure(1)
        ax = plt.subplot(111)
        bars, bottom  = [], 0
        for i in range(topn):
            #bar = plt.bar(np.arange(table.shape[1])+0.1,table.iloc[i],bottom=bottom,linewidth=0,color=itercolors.next(),hatch=iterhatches.next(),edgecolor=iteredgecolors.next())
            bar = plt.bar(np.arange(table.shape[1])+0.1,table.iloc[i],bottom=bottom,linewidth=0,color=next(itercolors),hatch=next(iterhatches),edgecolor=next(iteredgecolors))
            bars.append(bar)
            bottom = table.iloc[0:i+1].sum(axis=0)
        lgd = ax.legend(bars,list(table.index),loc='upper center',frameon=False,bbox_to_anchor=(0.5,-0.15),fontsize=8)
        ax.set_xticks(np.arange(table.shape[1])+0.5)
        ax.set_xticklabels(list(table.columns), rotation=90,horizontalalignment='center', size=xticklabelsize)
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.yaxis.set_ticks_position('left')
        ax.xaxis.set_ticks_position('bottom')
        plt.xlabel("Sample")
        plt.ylabel("Abundance")
        plt.xlim((0.0,table.shape[1]))
        if not raw:
            plt.ylim((0.0,1.0))
        fig.savefig(output_fn,bbox_extra_artists=(lgd,),dpi=300,bbox_inches='tight',format=fmt)

In [167]:
def convert_biom(input_biom):
    otu_table = os.path.join(tmp_dir,proj_name+'_otutab.txt')
    converted_biom = os.path.join(output_dir,proj_name+'_converted.biom')
    cmd = 'biom convert --table-type=\"OTU table\" -i %s -o %s --to-tsv --header-key taxonomy' % (input_biom,otu_table)
    #print (cmd)
    os.system(cmd)
    cmd = 'biom convert -i %s -o %s --to-json --process-obs-metadata taxonomy' % (otu_table,converted_biom)
    print (cmd)
    os.system(cmd)
    #cmd = 'biom validate-table -i %s' % (converted_biom)
    #os.system(cmd)
    return converted_biom

In [168]:
def micca2picrust(otutable="header[ref]+'otu.txt'"):
    ## aggregate duplicated OTU IDs ##
    cmd = 'R CMD BATCH --no-save --no-restore aggregate_duplicated_IDs.R'
    #print (cmd)
    os.system(cmd)
    ## get a summary statistics of an OTU-table ##
    #cmd = 'micca tablestats -i %sggotu_unique.txt -o %s/tablestats' % (input_dir,output_dir)
    cmd = 'micca tablestats -i %s%sotu_unique.txt -o %s/tablestats'  % (input_dir,header[ref],output_dir)
    os.system(cmd)
    ## get a minimum value of read depth to rarefy the OTU-table
    cmd = 'cat %s/tablestats/tablestats_samplesumm.txt > %s/statsummary.txt' % (output_dir,output_dir)
    #print (cmd)
    os.system(cmd)
    ## extract the minimum value ##
    rfn = os.path.join(output_dir,'statsummary.txt')
    fr = open(rfn,'r').readlines()
    depths = []
    fr.pop(0)
    for line in fr:
        tmp = line.split('\t')
        depths.append(int(tmp[1]))
    minval = min(depths)
    ## rarefy the OTU-table ##
    #cmd = 'micca tablerare -i %s/ggotu_unique.txt -o %s/otutable_rare.txt -d %d' % (input_dir,output_dir,minval)
    cmd = 'micca tablerare -i %s/%sotu_unique.txt -o %s/otutable_rare.txt -d %d' % (input_dir,header[ref],output_dir,minval)
    #print (cmd)
    os.system(cmd)
    ## taxonomic composition ##
    cmd = 'micca tabletotax -i %s/otutable_rare.txt -t %s -o %s/taxtables' % (output_dir,taxlist[ref],output_dir) 
    #print (cmd)
    os.system(cmd)
    ## Taxonomic composition bar plot ##
    for i in range(1,8):
        if is_eps=='yes':
            inputfile = '%s/taxtables/taxtable%d.txt' % (output_dir,i)
            outputfile = '%s/taxtables/taxtable%d.eps' % (output_dir,i)
            plotbars(inputfile,outputfile,fmt='eps')
            #cmd = 'micca tablebar -i %s/taxtables/taxtable%d.txt -f eps -o taxtables/taxtable%d.eps' % (i,output_dir,i)
        else:
            inputfile = '%s/taxtables/taxtable%d.txt' % (output_dir,i)
            outputfile = '%s/taxtables/taxtable%d.png' % (output_dir,i)
            plotbars(inputfile,outputfile,fmt='eps')
            #cmd = 'micca tablebar -i taxtables/taxtable%d.txt -o taxtables/taxtable%d.png' % (i,i)
        #print (cmd)
        #os.system(cmd)
    ## create an picrust-suited biom file ##
    converted_biom = os.path.join(output_dir,proj_name+'_converted.biom')
    cmd = 'micca tobiom -i %s/otutable_rare.txt -o %s' % (output_dir,converted_biom)
    #print (cmd)
    os.system(cmd)
    return converted_biom

In [169]:
def exec_picrust(converted_biom):
    normalized_biom = os.path.join(tmp_dir,proj_name+'_normalized.biom')
    premetagen_output = os.path.join(output_dir,proj_name+'_predicted_metagenomes.biom')
    catfun_output = os.path.join(output_dir,proj_name+'_catfun_KEGG_L3.txt')
    cmd = 'python /home/user/anaconda2/lib/python2.7/site-packages/PICRUSt-1.1.0-py2.7.egg/EGG-INFO/scripts/normalize_by_copy_number.py -i %s -o %s' % (converted_biom,normalized_biom)
    #print (cmd)
    os.system(cmd)
    cmd = 'python /home/user/anaconda2/lib/python2.7/site-packages/PICRUSt-1.1.0-py2.7.egg/EGG-INFO/scripts/predict_metagenomes.py -i %s -o %s' % (normalized_biom,premetagen_output)
    #print (cmd)
    os.system(cmd)
    cmd = 'python /home/user/anaconda2/lib/python2.7/site-packages/PICRUSt-1.1.0-py2.7.egg/EGG-INFO/scripts/categorize_by_function.py -i %s -f -c KEGG_Pathways -l 3 -o %s' % (premetagen_output,catfun_output)
    #print (cmd)
    os.system(cmd)
    picrust_outputs = [premetagen_output,catfun_output]
    return picrust_outputs

In [170]:
def exec_HUMAnN2(picrust_output):
    ## split a unified biom file into per-project biom files ##
    cmd = '/home/user/anaconda2/bin/humann2_split_table --input %s --output %s' % (picrust_output,tmp2_dir)
    #print (cmd)
    os.system(cmd)
    ## run HuMAnN2 ##
    os.chdir(tmp2_dir)
    bioms = glob.glob('*.biom')
    print (bioms)
    for biom in bioms:
        cmd = '/home/user/anaconda2/bin/humann2 --input %s --output %s --pathways-database %s' % (biom,tmp_dir,keggc)
        #print (cmd)
        os.system(cmd)
    os.chdir(input_dir)
    ## merge HUMAnN2 output ##
    abdtmp1 = os.path.join(tmp_dir,proj_name+'_abdtmp1.txt')
    cmd = '/home/user/anaconda2/bin/humann2_join_tables --input %s --output %s --file_name pathabundance' % (tmp_dir,abdtmp1)
    #print (cmd)
    os.system(cmd)
    ## Remove _Abundance ##
    abdtmp2 = os.path.join(tmp_dir,proj_name+'_abdtmp2.txt')
    cmd = 'sed "s/_Abundance//g" %s > %s' % (abdtmp1,abdtmp2)
    #print (cmd)
    os.system(cmd)
    ## Add full names to the KEGG pathway identifiers ##
    abd = os.path.join(output_dir,proj_name+'_abundance.txt')
    cmd = '/home/user/anaconda2/bin/humann2_rename_table -i %s -o %s -n kegg-pathway --simplify' % (abdtmp2,abd)
    #print (cmd)
    os.system(cmd)
    ## Normalize the table into relative abundances ##
    abdrel = os.path.join(output_dir,proj_name+'_abundance_relative.txt')
    cmd = '/home/user/anaconda2/bin/humann2_renorm_table -i %s -o %s --units relab' % (abd,abdrel)
    #print (cmd)
    os.system(cmd)
    ## merge HUMAnN2 output ##
    covtmp1 = os.path.join(tmp_dir,proj_name+'_covtmp1.txt')
    cmd = '/home/user/anaconda2/bin/humann2_join_tables --input %s --output %s --file_name pathcoverage' % (tmp_dir,covtmp1)
    #print (cmd)
    os.system(cmd)
    ## Remove _Coverage ##
    covtmp2 = os.path.join(tmp_dir,proj_name+'_covtmp2.txt')
    cmd = 'sed "s/_Coverage//g" %s > %s' % (covtmp1,covtmp2)
    #print (cmd)
    os.system(cmd)
    ## Add full names to the KEGG pathway identifiers ##
    cov = os.path.join(output_dir,proj_name+'_coverage.txt')
    cmd = '/home/user/anaconda2/bin/humann2_rename_table -i %s -o %s -n kegg-pathway --simplify' % (covtmp2,cov)
    #print (cmd)
    os.system(cmd)
    ## Normalize the table into relative abundances ##
    covrel = os.path.join(output_dir,proj_name+'_coverage_relative.txt')
    cmd = '/home/user/anaconda2/bin/humann2_renorm_table -i %s -o %s --units relab' % (cov,covrel)
    #print (cmd)
    os.system(cmd)
    HUMAnN2_outputs = [abd,abdrel,cov,covrel]
    return HUMAnN2_outputs

In [171]:
def exec_LEfSe(HUMAnN2_output):
    fr = open(HUMAnN2_output,'r').readlines()
    cd = open(class_def,'r').readlines()
    fr[0] = cd[0]
    HUMAnN2LEfSe = os.path.join(tmp_dir,'HUMAnN2LEfSe.txt')
    fw = open(HUMAnN2LEfSe,'w')
    for line in fr:
        fw.write(line)
    fw.close()
    ## format input file suitable to LEfSe ##
    tmp1 = os.path.join(tmp_dir,proj_name+'_LEfSe.in')
    cmd = 'python %s %s %s -c 1 -s -1 -u -1 -o 1000000' % (os.path.join(LEfSe,'format_input.py'),HUMAnN2LEfSe,tmp1)
    #print (cmd)
    os.system(cmd)
    ## execute LEfSe ##
    LEfSe_output = os.path.join(output_dir,proj_name+'_LEfSe.res')
    cmd = 'python %s %s %s' % (os.path.join(LEfSe,'run_lefse.py'),tmp1,LEfSe_output)
    #print (cmd)
    os.system(cmd)
    ## Draw cladogram ##
    fig_output = os.path.join(output_dir,proj_name+'_LEfSe.png')
    cmd = 'python %s %s %s --format png' % (os.path.join(LEfSe,'plot_cladogram.py'),LEfSe_output,fig_output)
    #print (cmd)
    os.system(cmd)
    return LEfSe_output

In [172]:
def exec_graphlan(LEfSe_output,title='\"Phylogenic tree\"'):
    tree_file = os.path.join(output_dir,proj_name+'_tree.txt')
    annotation_file = os.path.join(output_dir,proj_name+'_annotation.txt')
    outtree_file = os.path.join(output_dir,proj_name+'_outtree.xml')
    treefig_file = os.path.join(output_dir,proj_name+'_tree.png')
    ## export to graphlan from LEfSe & HUMAnN2 ##
    cmd = 'python %s -i %s -o %s -t %s -a %s --title %s --annotations 2,3 --external_annotations 4,5,6 --fname_row 0' % (os.path.join(graphlan,'export2graphlan/export2graphlan.py'),HUMAnN2_output,LEfSe_output,tree_file,annotation_file,title)
    #print (cmd)
    os.system(cmd)
    ## execute graphlan ##
    cmd = 'python %s --annot %s %s %s' % (os.path.join(graphlan,'graphlan_annotate.py'),annotation_file,tree_file,outtree_file)
    #print (cmd)
    os.system(cmd)
    ## Draw Tree ##
    cmd = 'python %s --dpi 300 --size 7.0 %s %s --external_legends' % (os.path.join(graphlan,'graphlan.py'),outtree_file,treefig_file)
    #print (cmd)
    os.system(cmd)

In [173]:
## start analysis ##
input_biom = os.path.join(input_dir,'input.biom')

In [174]:
## convert biom file for picrust ##
#if is_convert_biom=='yes':
#    converted_biom = convert_biom(input_biom)
#else:
#    converted_biom = os.path.join(input_dir,'input.biom')

In [175]:
otutable = os.path.join(input_dir,"header[ref]+'otu.txt'")
## convert biom file for picrust from micca ##
if is_micca2picrust=='yes':
    converted_biom = micca2picrust(otutable)
else:
    converted_biom = os.path.join(input_dir,'input.biom')

FileNotFoundError: File b'/Volumes/BUFFALO_4T/tanamai/16S/PRJEB9524/output/taxtables/taxtable1.txt' does not exist

In [ ]:
## picrust ##
if is_exec_picrust=='yes':
    picrust_outputs = exec_picrust(converted_biom)
    picrust_output = picrust_outputs[0]
else:
    picrust_output = os.path.join(output_dir,proj_name+'_predicted_metagenomes.biom')

In [ ]:
## HUMAnN2 ##
if is_exec_HUMAnN2=='yes':
    HUMAnN2_outputs = exec_HUMAnN2(picrust_output)
    HUMAnN2_output = HUMAnN2_outputs[1]
else:
    HUMAnN2_output = os.path.join(output_dir,proj_name+'_abundance_relative.txt')

In [ ]:
## LEfSe ##
if is_exec_LEfSe=='yes':
    LEfSe_output = exec_LEfSe(HUMAnN2_output)
else:
    LEfSe_output = os.path.join(output_dir,proj_name+'_LEfSe.res')

In [ ]:
## Graphlan ##
if is_exec_graphlan=='yes':
    exec_graphlan(LEfSe_output)